# CREM_Baselines
1. Just one round of CReM
2. Greedy Search: Take top 1000 molecules in training set, and continually re-apply CReM until surrogate value reaches a certain threshold.




In [1]:
from rdkit import Chem
# from rdkit.Chem import AllChem
# from rdkit.Chem import Draw, DataStructs
# from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
# from IPython.display import SVG, Image
IPythonConsole.molSize = (400,300)
IPythonConsole.ipython_useSVG=True

from crem.crem import mutate_mol

import os
import csv
import random
import numpy as np

import concurrent.futures

import torch
from torch_geometric.data import Batch

from src.utils.graph_utils import mol_to_pyg_graph
from src.utils.general_utils import load_surrogate_model, maybe_download_file

In [2]:
# Downloading files.
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
surrogate_model_url = "https://portal.nersc.gov/project/m3623/docking_score_models/NSP15_6W01_A_1_F/20201210_170_161_baseline/best_model.pth"
training_data_url = "https://portal.nersc.gov/project/m3623/MD/2col/NSP15_6W01_A_1_F.Orderable_zinc_db_enaHLL.2col.csv"

maybe_download_file(os.path.join(os.getcwd(), "train_data.csv"), training_data_url, "train_data")


train_data found.



In [3]:
# Loading Training Data and Surrogate Models
model = load_surrogate_model("..", surrogate_model_url, "", device)

with open(os.path.join(os.getcwd(), "train_data.csv"), 'r') as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    # List of lists, [[Smile, Score, row_number],...,]
    training_data = [[row[1], row[0]] for id, row in enumerate(reader)]

training_sample = random.sample(training_data, 1000)
molecules = [Chem.MolFromSmiles(row[0]) for row in training_sample]


Surrogate model found.

Surrogate model loaded


In [4]:
# Defining helper functions and threshold
threshold = -15

def get_scores(states, surrogate_model, device):
    g = Batch().from_data_list([mol_to_pyg_graph(state) for state in states])
    g = g.to(device)
    with torch.autograd.no_grad():
        pred_docking_score = surrogate_model(g, None)
    scores = pred_docking_score.numpy()
    return scores

def get_best(ms, model, device):
    # ms - list of molecules
    # model - surrogate_model
    pred = get_scores(ms, model, device)
    i = np.argmin(pred)
    return ms[i], pred[i]

def evolve_mol(mol):
    score = 0
    while score > threshold:
        new_mols = list(mutate_mol(Chem.AddHs(mol), db_name='replacements02_sc2.db', return_mol=True, min_size=0, max_size=8, min_inc=-3, max_inc=3, ncores=4))
        new_mols = [Chem.RemoveHs(i[1]) for i in new_mols]
        best_mol, score = get_best(new_mols + [mol], model, device)
        print('molecules generated:', len(new_mols))
        print('best score:', np.round(score, 3))
        mol = best_mol
    return best_mol, score

In [ ]:
# Greedy search
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(evolve_mol, molecules))

# best_mols = []
# best_scores = []
# for mol in molecule_smiles:
#     score = 0
#     while score > threshold:
#         new_mols = list(mutate_mol(Chem.AddHs(mol), db_name='replacements02_sc2.db', return_mol=True, min_size=0, max_size=8, min_inc=-3, max_inc=3, ncores=4))
#         new_mols = [Chem.RemoveHs(i[1]) for i in new_mols]
#         best_mol, score = get_best(new_mols + [mol], model, device)
#         print('molecules generated:', len(new_mols))
#         print('best score:', np.round(score, 3))
#         mol = best_mol
#     best_scores.append(score)
#     best_mols.append(mol)
#     break


molecules generated: 197
best score: -13.424
molecules generated: 127
best score: -11.213
molecules generated: 38
best score: -15.259
molecules generated: 120
best score: -11.29


Process ForkPoolWorker-20:3:
Process ForkPoolWorker-19:4:
Process ForkPoolWorker-19:1:
Process ForkPoolWorker-20:2:
Process ForkPoolWorker-19:2:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-24:3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ADChen/opt/anaconda3/envs/gcpn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-24:2:
  File "/Users/ADChen/opt/anaconda3/envs/gcpn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/ADChen/opt/anaconda3/envs/gcpn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/ADChen/opt/anaconda3/envs/gcpn/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/ADChen/opt/anaconda3/envs/gcpn/lib/python3.7/multiprocessing

In [27]:
for result in results:
    print(result)